In [ ]:
pip install keras streamlit pillow numpy h5py

In [ ]:
!pip -q install streamlit
!pip -q install pyngrok

In [6]:
!wget https://github.com/Coachnmomof3/UCB_COVID_Prediction_Model/blob/Gabriel_Cuchacovich/Models/VGG16_final_model/VGG16_model.h5

--2021-01-15 04:23:34--  https://github.com/Coachnmomof3/UCB_COVID_Prediction_Model/blob/Gabriel_Cuchacovich/Models/VGG16_final_model/VGG16_model.h5
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘VGG16_model.h5’

VGG16_model.h5          [ <=>                ]  87.62K  --.-KB/s    in 0.04s   

2021-01-15 04:23:34 (2.23 MB/s) - ‘VGG16_model.h5’ saved [89720]



In [ ]:
# Mounting the drive
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%%writefile app.py

import streamlit as st
import cv2
from PIL import Image, ImageOps
import numpy as np 
import tensorflow as tf

model_name = 'VGG16_model.h5'
model = tf.keras.models.load_model(model_name)

st.write("""
         # COVID-19 and Viral Pneumonia Predictions
         """
         )
st.write("This is a simple image classification web app to predict if a patient has COVID-19, Viral Pneumonia or is Healthy from chest X-Rays")
file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

def import_and_predict(image_data, model):
    size = (224,224)    
    image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = (cv2.resize(img, dsize=(224, 224),    interpolation=cv2.INTER_CUBIC))/255.
        
    img_reshape = img_resize[np.newaxis,...]
    
    prediction = model.predict(img_reshape)
        
    return prediction

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)
    
    max = np.max(prediction)
    
    if prediction[0][0]==max:
      st.text("The prediction is COVID")
    if prediction[0][1]==max:
      st.text("The prediction is NORMAL")
    if prediction[0][2]==max:
      st.text("The prediction is Viral Pneumonia")
    
    st.text("Likelihood 0: COVID, 1: NORMAL, 2: Viral Pneumonia")
    st.write(prediction)

Writing app.py


In [7]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 '/content/drive/MyDrive/Data Analysis Bootcamp/Resources/app.py' >/dev/null

Link to web app:
NgrokTunnel: "http://285a5c711d01.ngrok.io" -> "http://localhost:80"
2021-01-15 04:24:55.305682: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-15 04:24:57.166370: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-15 04:24:57.167606: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-15 04:24:57.177971: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-15 04:24:57.178026: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2e4f5eebcd18): /proc/driver/nvidia/version does not exist
2021-01-15 04:24:57.178873: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_device